### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{'_wandb': {'runtime': 5}},"{'exp_name': 'gpt_cont_video_nproj', 'override...",912lfl9s
1,{'_wandb': {'runtime': 56}},"{'env': {'game': 'Boxing', 'seed': 42, 'type':...",1qa0q7h6
2,{},"{'env': {'game': 'Boxing', 'seed': 42, 'type':...",1kdji3r4
3,{},"{'env': {'game': 'Boxing', 'seed': 42, 'type':...",24impp5c
4,{'_wandb': {'runtime': 2}},"{'env': {'game': 'Boxing', 'seed': 42, 'type':...",q70ywsra
...,...,...,...
1829,"{'loss': 0.7779913432076573, '_step': 117001, ...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
1830,"{'_wandb': {'runtime': 18252}, '_runtime': 182...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
1831,"{'loss': -0.9783781695365906, '_step': 117001,...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
1832,"{'negative_sim': 0.17696081340312958, 'pos_neg...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline'
exp_name = 'gpt_cont_video_nproj'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_13648/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
16,"{'neg_sim': 0.0025275754742324352, 'obs_loss':...","{'env': {'game': 'Seaquest', 'seed': 42, 'type...",1mf6593n,Seaquest
17,"{'best_metric_val': 0.22661915158897747, 'epoc...","{'env': {'game': 'UpNDown', 'seed': 42, 'type'...",2s8ksd8i,UpNDown
24,"{'act_f1': 0.2714593528976236, 'obs_loss': 0.1...","{'env': {'game': 'Qbert', 'seed': 42, 'type': ...",1hm845j7,Qbert
25,"{'_step': 234370, 'act_f1': 0.1136237603696462...","{'env': {'game': 'PrivateEye', 'seed': 42, 'ty...",23khce41,PrivateEye
26,"{'loss': 0.24047878453240484, 'act_f1': 0.2495...","{'env': {'game': 'RoadRunner', 'seed': 42, 'ty...",2maw2ltv,RoadRunner
27,"{'_timestamp': 1672084731.542855, 'loss': 0.05...","{'env': {'game': 'Pong', 'seed': 42, 'type': '...",1erl47tx,Pong
37,"{'obs_loss': 0.1381602907669415, 'reg_loss': 7...","{'env': {'game': 'MsPacman', 'seed': 42, 'type...",1ms3mlms,MsPacman
38,"{'mean_grad_norm': 0.009419828671002935, '_wan...","{'env': {'game': 'Krull', 'seed': 42, 'type': ...",5lhiqtei,Krull
39,"{'_wandb': {'runtime': 80549}, 'reg_loss': 7.2...","{'env': {'game': 'KungFuMaster', 'seed': 42, '...",62euq9ta,KungFuMaster
40,"{'act_f1': 0.22195873430357216, 'reg_loss': 5....","{'env': {'game': 'Kangaroo', 'seed': 42, 'type...",3jvagin3,Kangaroo


### 1. Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

Seaquest 1mf6593n
UpNDown 2s8ksd8i
Qbert 1hm845j7
PrivateEye 23khce41
RoadRunner 2maw2ltv
Pong 1erl47tx
MsPacman 1ms3mlms
Krull 5lhiqtei
KungFuMaster 62euq9ta
Kangaroo 3jvagin3
Jamesbond 1e9acxq0
Hero zjrlqthz
Gopher swqwann1
Frostbite 3n4i5s1v
Freeway 1x47xjh7
CrazyClimber 3sy7rbxs
DemonAttack 15itf50a
ChopperCommand 14fgek1a
BattleZone 1vdb4ej7
Breakout 2ruzsidr
Boxing huzyckzc
BankHeist 1qhibeet
Amidar 1t3d8oy5
Assault 216g5kjm
Asterix 2mxveu7b
Alien 1r79in30


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

> /tmp/ipykernel_13648/287021157.py(15)get_results_dict()
     13             import pdb
     14             pdb.set_trace()
---> 15             score = summary['eval_mean_traj_game_scores']
     16         except:
     17             continue



In [ ]:
data

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)